# Find the arteries

In [1]:
#: Our usual imports
import numpy as np  # Python array library
import matplotlib.pyplot as plt  # Python plotting library

The major arteries in a T1 MRI image often have high signal (white when
displaying in gray-scale).

Our task is to see if we can pick out the courses of the [vertebral, basilar](http://en.wikipedia.org/wiki/Vertebral_artery#mediaviewer/File:Vertebral_artery_3D_AP.jpg)
and [carotid](http://en.wikipedia.org/wiki/Internal_carotid_artery#mediaviewer/File:Magnetic_resonance_angiogram_of_segments_of_the_internal_carotid_artery.jpg)
arteries on this image.

The image is `ds107_sub001_highres.nii`.

Download this image as a file to the computer you're working on with:

In [2]:
# Run this cell.
import nipraxis
structural_fname = nipraxis.fetch_file('ds107_sub001_highres.nii')
structural_fname

This time we are going to load the image using the `nibabel` library.

In [3]:
#: Import nibabel
import nibabel as nib

Try loading the image using nibabel, to make an image object. Use tab
completion on `nib.` to see if you can find the function you need.  Go back to
have a look at [What is an
image?](https://textbook.nipraxis.org/what_is_an_image) if you are stuck.

In [4]:
#- Use nibabel to load the image ds107_sub001_highres.nii
img = nib.load(structural_fname)
# The resulting image object.
img

Now get the image array data from the nibabel image object.

**Hint**: You will need to use a **method** of the image object.

**Hint**: Don’t forget to use tab completion on the image object if you can’t
remember or don’t know the methods of the object.

**Hint**: The method you are looking for, gets floating point data from the
image.  If this hint isn't immediately helpful, check the hint above.

In [5]:
data = img.get_fdata()
# Show the data shape
data.shape

In [6]:
assert 'data' in dir()  # data variable exists.
assert data is not ...  # data changed from default value.
assert isinstance(data, np.ndarray)  # data is an array.
assert data.shape == (256, 256, 192)  # It is the correct shape.

Try plotting a few slices over the third dimension to see whether you can see
the arteries. For example, if `data` is your image array data, then you might
plot the *first* slice over the third dimension, with this code.

In [7]:
# Plotting a slice over the third dimension.
# Try changing the 0 here to some other slice number.
plt.imshow(data[:, :, 0], cmap='gray')

Now try looking for a good threshold so that you pick up only the very high
signal in the brain. A good place to start is to use `plt.hist` to get an idea
of the spread of values within the volume and within the slice.

In [8]:
#- Here you might try plt.hist or something else to find a threshold
data_1d = data.ravel()
plt.hist(data_1d, bins=100);

Try making a binarized image with your threshold and displaying slices with
that.

The code below uses a *comparison* operator to generate a Boolean array from
the floating point `data` array.  True in these arrays displays as white, and
False as black.

**For reflection**: With your threshold, what structures are you picking up?

In [9]:
#- Maybe display some slices from the data binarized with a threshold
threshold = 400
binarized_data = data > threshold
plt.imshow(binarized_data[:, :, 30], cmap='gray')

Now try taking a 3D subvolume out of the middle of the image.  Take the
approximate middle in all three axes.  Use this to pick out a good subvolume
of the image that still contains the big arteries.

You can create a subvolume by slicing the original `data` array to select a subset of the original data.  For example, here we take a subvolume of the data that starts at the pixels at position 10, and then takes all pixels up to position -10 (10 *before* the end of the dimension).  It does this in all three dimensions.  This knocks off the first 10 and the last 10 pixels in each dimension.

In [10]:
# Run this code.
# A subvolume of the original data.
# This shaves off 10 pixels from each side of the image.
my_subvolume = data[10:-10, 10:-10, 10:-10]
my_subvolume.shape

**Hint**: You will likely want to shave off more from the image, to make your
subvolume containing the arteries.

**Hint**: You may well want to shave off different amounts from different dimensions (axes) in the image.

**Hint**: See [the solution](arteries_solution.ipynb) if you get stuck.

In [11]:
#- Create a smaller 3D subvolume from the image data that still
#- contains the arteries. Replace the 10s, and -10s with better numbers.
subvolume = data[90:-90, 90:-90, 10:130]
# Show the shape of the subvolume.
subvolume.shape

Try binarizing the subvolume with some thresholds to see whether you can pick
out the arteries without much other stuff.

**Hint** — you might consider using the `np.percentile` function or `plt.hist`
to find a good threshold.

In [12]:
#- Try a few plots of binarized slices and other stuff to find a good
#- threshold.
pct_99 = np.percentile(subvolume, 99)
binarized_subvolume = subvolume > pct_99
# Show the binarized subvolume.
plt.imshow(binarized_subvolume[:, :, 20], cmap='gray')

If you have a good threshold and a good binarized subset, see if you can see
the arterial structure using the following fancy function to plot the
binarized image with a 3D rendering.  To use this function, you will need to
install the [scikit-image](http://scikit-image.org/) toolbox.  First see if
scikit-image is installed with the command `import skimage` from the Python /
IPython console.  If this gives you an `ImportError`, then open a *new
terminal window* and install scikit-image with:

```
!pip3 install --user scikit-image
```

Careful – do *not* run this `pip3 install` command from Python / IPython /
Jupyter, but from the terminal command window.

When you have done the scikit-image install, continue with this code.

In [13]:
from skimage import measure

With that import done, here is the fancy function to display your subvolume in
3D:

In [14]:
#: This function uses matplotlib 3D plotting and sckit-image for
# rendering
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def binarized_surface(binary_array):
    """ Do a 3D plot of the surfaces in a binarized image

    The function does the plotting with scikit-image and some fancy
    commands that we don't need to worry about at the moment.
    """
    # Here we use the scikit-image "measure" function
    verts, faces, _, _ = measure.marching_cubes(binary_array, 0)
    fig = plt.figure(figsize=(10, 12))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0, alpha=0.5)
    ax.add_collection3d(mesh)
    ax.set_xlim(0, binary_array.shape[0])
    ax.set_ylim(0, binary_array.shape[1])
    ax.set_zlim(0, binary_array.shape[2])

For example, let’s say you have a binarized subvolume of the original
data called `binarized_subvolume`. You could do a 3D rendering of this
binary image with:

In [15]:
binarized_surface(binarized_subvolume)